In [139]:
import iexfinance.stocks as iex
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

In [140]:
# Change as you please (note that by-minute data only includes past three months...)
start_date = datetime(2015,12,31)
end_date   = datetime(2019,3,2)

# Simple function that returns respective time-series given a stock ticker
def ticker_to_series(ticker='', sd=None, ed=None):
    if ticker == '':
        return
    if sd is None or ed is None:
        return
    # Default list of dropped columns from iexfinance's data
    to_drop = ['high', 'low', 'open', 'volume']
    # Retrieve data
    t_df = iex.get_historical_data(ticker, sd, ed, 
                                   output_format='pandas').drop(columns=to_drop)
    return t_df

# Similar functionality, but by the minute (up to last three months)
def ticker_to_series_minute(ticker='', sd=None, ed=None):
    if ticker == '':
        return
    # Default list of dropped columns from iexfinance's data
    to_drop = ['high', 'low', 'open', 'volume', 'average', 'changeOverTime',
               'date', 'label', 'marketAverage', 'marketChangeOverTime',
               'marketClose', 'marketHigh', 'marketLow', 'marketNotional',
               'marketNumberOfTrades', 'marketOpen', 'marketVolume', 'notional',
               'numberOfTrades']
    # By minute data only available for query on specific day input...
    t_delt = ed - sd # calculate "change in time"
    
    # i = 0:
    t_df = t2s_min_helper(ticker, sd)
    # i > 0:
    for i in range(1,t_delt.days+1):
        t_df_i = t2s_min_helper(ticker, sd+timedelta(i))
        t_df = t_df.append(t_df_i)
    
    return t_df.drop(columns=to_drop)

# Helper function
def t2s_min_helper(ticker, d):
    # Retrieve data
    t = iex.get_historical_intraday(ticker, d, output_format='pandas')
    return t

In [141]:
# Examples
micron_byday = ticker_to_series('MU', start_date, end_date)
micron_bymin = ticker_to_series_minute('MU', datetime(2019,2,4), end_date)

print(micron_bymin.shape)
micron_bymin.head()
micron_bymin.tail()

(7410, 1)


,close
2019-03-01 15:55:00,41.55
2019-03-01 15:56:00,41.53
2019-03-01 15:57:00,41.53
2019-03-01 15:58:00,41.56
2019-03-01 15:59:00,41.59


In [142]:
import plotly.plotly as py
import plotly.graph_objs as go

# Plotly sign-in
py.sign_in('cjk2bg', 'UJ3eIEBVJEf5EXa8zxCb')

data = [
    go.Scatter(x=micron_bymin.index,
               y=micron_bymin['close'])
]
layout = go.Layout(
    title='Micron Technology (MU) Stock Price since February 4th, 2019',
    xaxis=dict(type="category", 
               showticklabels=False,
               zeroline=True,
               zerolinewidth=1),
    yaxis=dict(zeroline=True,
               zerolinewidth=1)
)
fig = go.Figure(data=data,layout=layout)

py.iplot(fig)